In [1]:
# Your objective is to analyze the data to show how 
#four treatments (Capomulin, Infubinol, Ketapril, and Placebo) compare

#Creating a scatter plot that shows how the tumor volume 
#changes over time for each treatment.

#Creating a scatter plot that shows how the number of metastatic 
#(cancer spreading) sites changes over time for each treatment.

#Creating a scatter plot that shows the number of mice still alive 
#through the course of treatment (Survival Rate)

#Creating a bar graph that compares the total % tumor volume 
#change for each drug across the full 45 days.

In [2]:
%matplotlib notebook

In [3]:
#Import libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sb

In [4]:
data_csv = "Resources/clinicaltrial_data.csv"
mouse_csv = "Resources/mouse_drug_data.csv"

In [5]:
data_df = pd.read_csv(data_csv)
mouse_df = pd.read_csv(mouse_csv)

In [6]:
#verifying data was read in correctly
data_df.head()

Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites
0     b128          0                45.0                 0
1     f932          0                45.0                 0
2     g107          0                45.0                 0
3     a457          0                45.0                 0
4     c819          0                45.0                 0

In [7]:
#verifying data was read in correctly
mouse_df.head()

Mouse ID      Drug
0     f234  Stelasyn
1     x402  Stelasyn
2     a492  Stelasyn
3     w540  Stelasyn
4     v764  Stelasyn

In [8]:
# make one big table & check to see if there are errors
bigtable_df = pd.merge(data_df, mouse_df, on='Mouse ID')

#count how many 
bigtable_df['Drug'].value_counts()


Capomulin    230
Ramicane     228
Stelasyn     194
Ketapril     188
Naftisol     186
Zoniferol    182
Placebo      181
Ceftamin     178
Infubinol    178
Propriva     161
Name: Drug, dtype: int64

In [9]:
new_bigtable_df = bigtable_df.set_index("Drug")
new_bigtable_df.head()

Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites
Drug                                                               
Capomulin     b128          0           45.000000                 0
Capomulin     b128          5           45.651331                 0
Capomulin     b128         10           43.270852                 0
Capomulin     b128         15           43.784893                 0
Capomulin     b128         20           42.731552                 0

In [10]:
Capomulin_df = new_bigtable_df.loc['Capomulin',['Mouse ID', 'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites']]

Mouse_ID = Capomulin_df['Mouse ID']
Mouse_ID = Mouse_ID.value_counts()
Mouse_ID_df = pd.DataFrame(Mouse_ID)
Mouse_ID_df = Mouse_ID_df.reset_index()
Mouse_ID_df = Mouse_ID_df.rename(index=str, columns={ "Mouse ID": "Count"})
Mouse_ID_df['Start Volume'] = 0.0
Mouse_ID_df['Start Met Size'] = 0.0

Capomulin_df= Capomulin_df.reset_index()

#for i, row in Mouse_ID_df.iterrows():
    #value = Capomulin_df.loc[Capomulin_df['Mouse ID'] == Mouse_ID_df['index'][i], 'Tumor Volume (mm3)'].max()
    #Mouse_ID_df.at[i,'max'] = value
    
for i, row in Mouse_ID_df.iterrows():
    value = Mouse_ID_df['index'][i]
    
    for w, row in Capomulin_df.iterrows():
        Q = Capomulin_df['Mouse ID'][w]
        if((Q == value) and (Capomulin_df['Timepoint'][w] == 0)):
            StartVol = Capomulin_df['Tumor Volume (mm3)'][w]
            StartMet = Capomulin_df['Metastatic Sites'][w]
        

    #print(value)
    Mouse_ID_df.at[i,'Start Volume'] = StartVol
    Mouse_ID_df['Start Met Size'] = StartMet

Mouse_ID_df.head()

index  Count  Start Volume  Start Met Size
0  j119     10          45.0               0
1  y793     10          45.0               0
2  m957     10          45.0               0
3  w914     10          45.0               0
4  s185     10          45.0               0

In [29]:
Capomulin_df['Volume Change'] = 0.0
Capomulin_df['Met. Change'] = 0.0

for i, row in Capomulin_df.iterrows():
    IDValue = Capomulin_df.loc[i,'Mouse ID']
    
    for w, row in Mouse_ID_df.iterrows():
        if Mouse_ID_df.loc[w,'index'] == IDValue:
            StartVolume = Mouse_ID_df.loc[w,'Start Volume']
            StartMet = Mouse_ID_df.loc[w,'Start Met Size'] 
            
    Volume = Capomulin_df.loc[i, 'Tumor Volume (mm3)']
    Met = Capomulin_df.loc[i, 'Metastatic Sites']
    
    Capomulin_df.at[i,'Volume Change'] = StartVolume - Volume
    Capomulin_df.at[i,'Met. Change'] = Met - StartMet
    
Capomulin_df.head()


Drug Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites  \
0  Capomulin     b128          0           45.000000                 0   
1  Capomulin     b128          5           45.651331                 0   
2  Capomulin     b128         10           43.270852                 0   
3  Capomulin     b128         15           43.784893                 0   
4  Capomulin     b128         20           42.731552                 0   

   Volume Change  Met. Change  
0       0.000000          0.0  
1      -0.651331          0.0  
2       1.729148          0.0  
3       1.215107          0.0  
4       2.268448          0.0

In [51]:
days = np.arange(0,50,5)

C_Mouse_Alive = Capomulin_df.loc[:,['Mouse ID', 'Timepoint']]

C_Mouse_Alive=C_Mouse_Alive.set_index("Timepoint")
C_Mouse_Alive=C_Mouse_Alive.reset_index()

Count = C_Mouse_Alive['Timepoint'].value_counts()

C_Alive_Days_df = pd.DataFrame(Count)
C_Alive_Days_df = C_Alive_Days_df.reset_index()
C_Alive_Days_df = C_Alive_Days_df.sort_values(by='index', ascending = True)
C_Alive_Days_df.head(10)


index  Timepoint
2      0         25
1      5         25
0     10         25
3     15         24
4     20         23
7     25         22
6     30         22
5     35         22
9     40         21
8     45         21

In [56]:
Drugs_df = new_bigtable_df.loc[['Infubinol', 'Ketapril', 'Placebo'],
                               ['Mouse ID', 'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites']]

#Mouse_ID = Capomulin_df['Mouse ID']
#Mouse_ID = Mouse_ID.value_counts()
#Mouse_ID_df = pd.DataFrame(Mouse_ID)
#Mouse_ID_df = Mouse_ID_df.reset_index()
#Mouse_ID_df = Mouse_ID_df.rename(index=str, columns={ "Mouse ID": "Count"})
#Mouse_ID_df['Start Volume'] = 0.0
#Mouse_ID_df['Start Met Size'] = 0.0

Drugs_df.head(10000)



Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites
Drug                                                               
Infubinol     c326          0           45.000000                 0
Infubinol     c326          5           49.340203                 0
Infubinol     c139          0           45.000000                 0
Infubinol     c139          5           48.753112                 0
Infubinol     c139         10           49.992954                 0
Infubinol     c139         15           51.833225                 0
Infubinol     c139         20           56.742007                 0
Infubinol     c139         25           61.461970                 0
Infubinol     c139         30           64.149871                 1
Infubinol     c139         35           67.821420                 1
Infubinol     c139         40           69.428141                 2
Infubinol     c139         45           72.226731                 2
Infubinol     v339          0           45.000000                 0
Infubinol     v339          5           46.250112                 0
Infubinol     a577          0           45.000000                 0
Infubinol     a577          5           46.197720                 1
Infubinol     a577         10           49.509639                 1
Infubinol     a577         15           50.915007                 2
Infubinol     a577         20           52.365185                 2
Infubinol     a577         25           54.380932                 2
Infubinol     a577         30           57.031862                 2
Infubinol     y163          0           45.000000                 0
Infubinol     y163          5           48.284970                 0
Infubinol     y163         10           49.112399                 1
Infubinol     y163         15           51.523068                 1
Infubinol     y163         20           54.571076                 1
Infubinol     y163         25           56.176385                 1
Infubinol     y163         30           59.412537                 2
Infubinol     y163         35           61.730196                 2
Infubinol     y163         40           63.620680                 2
...            ...        ...                 ...               ...
Placebo       t718         25           55.876588                 3
Placebo       t718         30           57.704114                 3
Placebo       t718         35           60.463899                 3
Placebo       t718         40           61.530399                 3
Placebo       t718         45           63.423695                 4
Placebo       o302          0           45.000000                 0
Placebo       o302          5           48.937012                 0
Placebo       o302         10           49.701450                 0
Placebo       o302         15           51.381489                 0
Placebo       o302         20           56.177611                 0
Placebo       o302         25           60.517103                 0
Placebo       o302         30           62.030594                 1
Placebo       i477          0           45.000000                 0
Placebo       i477          5           46.657385                 0
Placebo       i477         10           47.520632                 0
Placebo       c757          0           45.000000                 0
Placebo       c757          5           47.783280                 0
Placebo       c757         10           49.219317                 0
Placebo       c757         15           51.532603                 0
Placebo       c757         20           52.818634                 1
Placebo       c757         25           57.106418                 2
Placebo       c757         30           60.098572                 2
Placebo       c757         35           64.291164                 2
Placebo       c757         40           66.604368                 2
Placebo       c757         45           69.042841                 3
Placebo       t994          0           45.000000                 